In [39]:
#preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pandas_profiling import ProfileReport

#model
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix

from pycaret.classification import *

from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
import xgboost as xgb
from pycaret.classification import *

Exception ignored in: <function tqdm.__del__ at 0x7f4e2fd095f0>
Traceback (most recent call last):
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x7f4e2fd095f0>
Traceback (most recent call last):
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


In [36]:
data_o = pd.read_csv('/work/patient-readmitted-prdiction/train_dataset.csv')
data_y = pd.read_csv('/work/patient-readmitted-prdiction/train_label.csv')
test = pd.read_csv('/work/patient-readmitted-prdiction/test_dataset.csv')

data = pd.concat([data_o,test])
data = data.replace('?', np.nan)
data.reset_index(drop=True, inplace= True)

### EDA
Do the Exploring Data Analysis by using "Pandas Reprot" 

In [33]:
#profile = ProfileReport(data, title = 'patient_readmitted_prediction')
#profile.to_file("your_report.html")

### FEATURE ENGINEER

In [3]:
## age, gender, race preprocess, delete weight, payer_code, medical_specialty
def basic_agw(data):
    data_o = data

    ## age --> Label Encoding
    for i in range(0,10):
        data_o['age'] = data_o['age'].replace('['+str(10*i)+'-'+str(10*(i+1))+')', i+1)

    ## gender --> delete the unknown gender and label encoding (only 3 of them are NA)
    fordrop = data_o[data_o['gender'] == 'Unknown/Invalid'].index
    data_o.drop(index = fordrop, axis = 0, inplace = True)
    data_o.reset_index(drop = True, inplace = True)
    data_o['gender'] = data_o['gender'].replace({'Male':1,'Female':0})

    ## weight --> there are 96.8% of data are N/A, so I delete this feature
    #print('N/A percentage of weight:', data_o['weight'].value_counts()['?']/len(data_o))
    data_o.drop(['weight','payer_code','medical_specialty'], axis=1, inplace = True)

    ## race -> I couldn't find out the significant difference on readmitted between each races,
    ##         So i just do the basic one-hot encoding, and only left Caucasian, Afircan American. 
    data_o['race'].fillna('Other', inplace = True)
    data_o['race'].replace({'Asian':'Other','Hispanic':'Other'}, inplace=True)
    data_o = data_o.join(pd.get_dummies(data_o.race))
    data_o.drop(['race','gender'], axis=1 , inplace = True)

    # Service_utilization
    data_o['service_utilization'] = data_o['number_outpatient'] + data_o['number_emergency'] + data_o['number_inpatient']
    return data_o


data = basic_agw(data)

In [4]:
def med_times(data):
    keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide.metformin', 'tolazamide', 'metformin.pioglitazone','metformin.rosiglitazone', 'glimepiride.pioglitazone', 'glipizide.metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
    for col in keys:
        colname = str(col) + 'temp'
        data[colname] = data[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
        data['numchange'] = 0

    for col in keys:
        colname = str(col) + 'temp'
        data['numchange'] += data[colname]
        del data[colname]

    return data

data = med_times(data)
    

In [5]:
def medstatus(data):
    data_o = data
    #A1Cresult, max_glu_serum  -->  adjust to 1,0
    #A1Cresult --> None, Norm = 0, >7>8 = 1
    
    data_o['A1Cresult'] = data_o['A1Cresult'].replace({'None':-10,'Norm':0, '>7':10, '>8':10})

    #max_glu_serum --> None, Norm =>0 , >200, >300 to 1, 0
    data_o['max_glu_serum'] = data_o['max_glu_serum'].replace({'None':-10,'Norm':0, '>200':10, '>300':10})

    ## delete the columns with toooo many No => 
    ## acetohexamide, tolbutamide, troglitazone, tolazamide, glipizide.metformin, glimepiride.pioglitazone, metformin.rosiglitazone, metformin.pioglitazone]

    tomanyno = ['acetohexamide','tolbutamide','acarbose','miglitol','troglitazone','examide','citoglipton'
    ,'glipizide.metformin','glimepiride.pioglitazone','metformin.rosiglitazone','metformin.pioglitazone','metformin','repaglinide','nateglinide'
    ,'chlorpropamide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','tolazamide','glyburide.metformin']
    data_o.drop(tomanyno, axis=1, inplace = True)

    data_o['insulin'] = data_o['insulin'].replace({'No':0,'Steady':0,'Down':1,'Up':1})
    data_o['change'] = data_o['change'].replace({'No':0,'Ch':1})
    data_o['diabetesMed'] = data_o['diabetesMed'].replace({"Yes":1,"No":0})

    data_o = pd.get_dummies(data_o, columns=['A1Cresult','max_glu_serum'])

    return data_o

data = medstatus(data)

In [6]:
## DIAG series
def diag_series(data):
    data_o = data
    data_o.fillna('-1', inplace = True)
    data_o['level1_diag1'] = data_o['diag_1']

    data_o.loc[data_o['diag_1'].str.contains('V'), ['level1_diag1']] = 0
    data_o.loc[data_o['diag_1'].str.contains('E'), ['level1_diag1']] = 0

    data_o['level1_diag1'] = data_o['level1_diag1'].astype(float)

    for index, row in data_o.iterrows():
        if (row['level1_diag1'] >= 390 and row['level1_diag1'] < 460) or (np.floor(row['level1_diag1']) == 785):
            data_o.loc[index, 'level1_diag1'] = 1
        elif (row['level1_diag1'] >= 460 and row['level1_diag1'] < 520) or (np.floor(row['level1_diag1']) == 786):
            data_o.loc[index, 'level1_diag1'] = 2
        elif (row['level1_diag1'] >= 520 and row['level1_diag1'] < 580) or (np.floor(row['level1_diag1']) == 787):
            data_o.loc[index, 'level1_diag1'] = 3
        elif (np.floor(row['level1_diag1']) == 250):
            data_o.loc[index, 'level1_diag1'] = 4
        elif (row['level1_diag1'] >= 800 and row['level1_diag1'] < 1000):
            data_o.loc[index, 'level1_diag1'] = 5
        elif (row['level1_diag1'] >= 710 and row['level1_diag1'] < 740):
            data_o.loc[index, 'level1_diag1'] = 6
        elif (row['level1_diag1'] >= 580 and row['level1_diag1'] < 630) or (np.floor(row['level1_diag1']) == 788):
            data_o.loc[index, 'level1_diag1'] = 7
        elif (row['level1_diag1'] >= 140 and row['level1_diag1'] < 240):
            data_o.loc[index, 'level1_diag1'] = 8
        else:
            data_o.loc[index, 'level1_diag1'] = 0


    data_o.drop(['diag_1','diag_2','diag_3'], axis=1, inplace = True)

    return data_o


data = diag_series(data)

In [7]:
# CALCULATE THE PATIENTS

def manytimes_patients(data):
    a = data['patient_nbr'].value_counts()
    biggerthan2 = a[a>=2].index
    data['duplicates_patient'] = 0
    for i in range(len(data)):
        if data['patient_nbr'][i] in biggerthan2:
            data['duplicates_patient'][i] = 1

    return data

def manytimes_patients2(data):
    aa = dict(data['patient_nbr'].value_counts())
    data['duplicates_patient'] = 0
    for i in range(len(data)):
        data['duplicates_patient'][i] = aa[data['patient_nbr'][i]]

    return data


data = manytimes_patients2(data)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### TRAINing MODEL

In [8]:
data = pd.merge(data, data_y, how='left', on='encounter_id')

#MIN_MAX_SCALER
num_feature = ['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses','numchange','duplicates_patient']
scaler = MinMaxScaler()
num_data = data[num_feature]
data[num_feature] = scaler.fit(num_data)
data[num_feature] = scaler.transform(num_data)

data.drop(['encounter_id','patient_nbr'], axis=1, inplace = True)

In [9]:
X = data.iloc[:71409]
y = data['readmitted'].iloc[:71409]
X.drop('readmitted', axis =1, inplace = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
xgboost = XGBClassifier(objective = 'multi:softprob',eval_metric = 'mlogloss', num_class=3 )
xgboost.fit(X_train, y_train)
preds = xgboost.predict(X_test)
print('Accuracy:', accuracy_score(preds, y_test))

/usr/local/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
Accuracy: 0.7009522475843719


### SUBMISSION

In [11]:
def to_sub(y_test_pred):
    testt = pd.read_csv('/home/jovyan/work/test_dataset.csv')
    encounter_id = testt['encounter_id']
    submission = pd.DataFrame()
    submission['encounter_id'] = encounter_id
    submission['readmitted'] = y_test_pred
    submission.to_csv('submission1.csv', index = False)

to_sub(y_test_pred)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b0119b9f-bb77-485b-97b3-c2bdea5c1634' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>